In [ ]:
#Set up

In [119]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [ ]:
#Set the path of the ChromeDrive

In [105]:
chrome_driver_path = 'D:/Program Files (x86)/path/chromedriver.exe'  # use the real path of your own 
service = Service(chrome_driver_path)

In [21]:
#Open the Chrome

SyntaxError: invalid syntax (400094230.py, line 1)

In [121]:
options = Options()
options.binary_location = r"D:\Program Files\chrome-win64\chrome.exe"
options.add_argument('--headless')# You can delete the row for headless running mode 
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
#Open the target URL

In [123]:
url = 'https://minasidor.sssb.se/en/available-apartments/'
driver.get(url)
print("开始等待页面加载")
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.PaginationNextExists > a')))
print("页面加载完成")

开始等待页面加载
页面加载完成


In [ ]:
#IF the Cookie bump out

In [125]:
try:
    accept_cookie_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@class='cc_btn cc_btn_accept_all' and text()='I accept']"))
    )
    accept_cookie_button.click()
    print("已接受cookie")
except (NoSuchElementException, TimeoutException):
    print("未找到cookie弹窗或点击失败")

data = []

未找到cookie弹窗或点击失败


In [ ]:
#Access to the web content

In [127]:
def clean_text(text):
    try:
        # html实体解码 + 编码解码防乱码
        return html.unescape(text).encode('latin1').decode('utf-8')
    except:
        return text  # 如果出错就返回原始文本

def get_page_data():
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    apt_blocks = soup.find_all('div', class_='column medium-3')

    # 提取详情数据区
    details_headers = soup.find_all('ul', class_='apt-list apt-details-headers')
    details_data = soup.find_all('ul', class_='apt-list apt-details-data')

    for i, block in enumerate(apt_blocks):
        # 房型名称
        room_type_tag = block.find('h2', class_='apt-title')
        room_type = clean_text(room_type_tag.get_text(strip=True)) if room_type_tag else ''

        # 地址
        address_tag = block.find('p', class_='apt-address')
        address = clean_text(address_tag.get_text(strip=True)) if address_tag else ''

        # 租期信息
        rent_type_tags = block.find_all('p', class_='rent-type')
        rent_type = ' | '.join([clean_text(t.get_text(strip=True)) for t in rent_type_tags])

        # 详情数据
        if i < len(details_data):
            detail_li = details_data[i].find_all('li')
            area = clean_text(detail_li[0].get_text(strip=True)) if len(detail_li) > 0 else ''
            living_space = clean_text(detail_li[1].get_text(strip=True)) if len(detail_li) > 1 else ''
            rent = clean_text(detail_li[2].get_text(strip=True)) if len(detail_li) > 2 else ''
            moving_in = clean_text(detail_li[3].get_text(strip=True)) if len(detail_li) > 3 else ''
            queue_days = clean_text(detail_li[5].get_text(strip=True)) if len(detail_li) > 5 else ''
            floor = clean_text(detail_li[6].get_text(strip=True)) if len(detail_li) > 6 else ''
        else:
            area = living_space = rent = moving_in = queue_days = floor = ''

        data.append({
            'Room Type': room_type,
            'Address': address,
            'Rent Type': rent_type,
            'Area': area,
            'Living Space': living_space,
            'Rent': rent,
            'Moving In': moving_in,
            'Queue Days': queue_days,
            'Floor': floor,
        })

# 调用函数
data = []
get_page_data()

# 打印结果看效果
for item in data:
    print(item)

{'Room Type': 'Corridor room', 'Address': 'Studentbacken 21 / 1212', 'Rent Type': '10 mån hyra | Electricity included', 'Area': 'Jerum', 'Living Space': '17 m²', 'Rent': '4\xa0805 SEK', 'Moving In': '2025-09-16', 'Queue Days': '90 (4st)', 'Floor': '2'}
{'Room Type': 'Corridor room', 'Address': 'Studentbacken 21 / 1410', 'Rent Type': '10 mån hyra | Electricity included', 'Area': 'Jerum', 'Living Space': '17 m²', 'Rent': '4\xa0593 SEK', 'Moving In': '2025-09-16', 'Queue Days': '90 (4st)', 'Floor': '4'}
{'Room Type': 'Corridor room', 'Address': 'Studentbacken 21 / 1603', 'Rent Type': '10 mån hyra | Electricity included', 'Area': 'Jerum', 'Living Space': '17 m²', 'Rent': '4\xa0593 SEK', 'Moving In': '2025-09-16', 'Queue Days': '90 (4st)', 'Floor': '6'}
{'Room Type': '3 rooms & kitchen', 'Address': 'Norra Stationsgatan 97 / 1101', 'Rent Type': 'Electricity included', 'Area': 'Idun', 'Living Space': '80 m²', 'Rent': '11\xa0203 SEK', 'Moving In': '2025-09-01', 'Queue Days': '1306 (6st)', 'Flo

In [ ]:
#Find the next page

In [129]:
# 初始化变量以存储上一个页面的 URL
previous_url = driver.current_url

# 循环点击下一页按钮并获取数据
while True:
    try:
        # 找到并点击下一页按钮
        next_button = driver.find_element(By.CSS_SELECTOR, 'span.PaginationNextExists > a')
        print("找到下一页按钮")
        
        # 移动到元素并点击
        action = ActionChains(driver)
        action.move_to_element(next_button).click().perform()
        print("点击下一页按钮")
        
        # 等待页面加载完成并检查URL是否变化
        WebDriverWait(driver, 20).until(lambda driver: driver.current_url != previous_url)
        print("页面加载完成")
        
        # 更新previous_url
        previous_url = driver.current_url
        
        # 等待更多时间确保页面完全加载
        time.sleep(10)
        
        # 获取当前页的数据
        get_page_data()
        print("获取当前页数据")
    except (NoSuchElementException, ElementClickInterceptedException, TimeoutException) as e:
        print(f"异常: {e}")
        # 如果没有下一页按钮，跳出循环
        break

# 关闭浏览器
driver.quit()

# 创建DataFrame
df = pd.DataFrame(data)

        

找到下一页按钮
点击下一页按钮
页面加载完成
获取当前页数据
找到下一页按钮
点击下一页按钮
页面加载完成
获取当前页数据
找到下一页按钮
点击下一页按钮
页面加载完成
获取当前页数据
找到下一页按钮
点击下一页按钮
页面加载完成
获取当前页数据
找到下一页按钮
点击下一页按钮
页面加载完成
获取当前页数据
找到下一页按钮
点击下一页按钮
页面加载完成
获取当前页数据
找到下一页按钮
点击下一页按钮
页面加载完成
获取当前页数据
异常: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.PaginationNextExists > a"}
  (Session info: chrome=139.0.7258.66); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff77cde6b55+79621]
	GetHandleVerifier [0x0x7ff77cde6bb0+79712]
	(No symbol) [0x0x7ff77cb7c0ea]
	(No symbol) [0x0x7ff77cbd2f56]
	(No symbol) [0x0x7ff77cbd320c]
	(No symbol) [0x0x7ff77cc265b7]
	(No symbol) [0x0x7ff77cbfb17f]
	(No symbol) [0x0x7ff77cc233d0]
	(No symbol) [0x0x7ff77cbfaf13]
	(No symbol) [0x0x7ff77cbc4151]
	(No symbol) [0x0x7ff77cbc4ee3]
	GetHandleVerifier [0x0x7ff77d0a686d+2962461]
	GetHandleVerifier [0x0x7ff77d0a0b8d+29

In [ ]:
#Output

In [131]:
# 打印DataFrame
print(df)

# 保存到CSV文件
df.to_csv("output.csv", encoding="utf-8-sig", index=False)

            Room Type                        Address  \
0       Corridor room        Studentbacken 21 / 1212   
1       Corridor room        Studentbacken 21 / 1410   
2       Corridor room        Studentbacken 21 / 1603   
3   3 rooms & kitchen  Norra Stationsgatan 97 / 1101   
4   4 rooms & kitchen          Röntgenvägen 1 / 2219   
..                ...                            ...   
82                                                     
83                                                     
84      Corridor room          Armégatan 32 A / 1901   
85                                                     
86                                                     

                                           Rent Type          Area  \
0                 10 mån hyra | Electricity included         Jerum   
1                 10 mån hyra | Electricity included         Jerum   
2                 10 mån hyra | Electricity included         Jerum   
3                               Electricity inc